### Load the train dataset.

In [76]:
import os
import random
import pandas as pd

news = []

for directory in os.listdir("1150haber/raw_texts/"):
    for filename in os.listdir("1150haber/raw_texts/" + directory):
        f = open("1150haber/raw_texts/" + directory + "/" + filename, "r", encoding="windows-1254")
        new = f.read().replace("\n", " ")
        new = new.lower()
        news.append(new)
        
random.shuffle(news)
print(len(news))

data = {"Haber": news}
df_train = pd.DataFrame(data)
df_train["Haber"] = df_train["Haber"].apply(str)
df_train.sample(20)

920


,Haber
826,ankara trafik vakfı tarafından yayınlanan traf...
176,'yosma' ile bir hafta sonu koç holding...
823,"enflasyon sepetinden tornavida çıkıyor, ekmek ..."
62,güvenlik konseyi kıbrıs'ta bölündü bm güvenl...
241,hakan peker 'taşgibi' peker'in albüm fotoğ...
505,bağırsak kanserini önlemede röntgen teknolojis...
2,luce’ni̇n i̇syani zago’nun pozisyon hatası y...
695,göz göre göre terim’in inanılmaz kredisi gal...
221,milli takım cnn’de lider ünlü haber kanalını...
699,beş farklı masaldan esinlenilerek çekilen ‘anl...


### Load the test dataset.

In [77]:
import os
import random
import pandas as pd

news = []
labels = []
filenames = []
i = 0

for directory in os.listdir("1150haber/test/"):
    i += 1
    for filename in os.listdir("1150haber/test/" + directory):
        f = open("1150haber/test/" + directory + "/" + filename, "r", encoding="windows-1254")
        new = f.read().replace("\n", " ")
        new = new.lower()
        news.append(new)
        labels.append(i)
        
print(len(news))

data = {"Haber": news, "Sınıf": labels}
df_test = pd.DataFrame(data)
df_test["Haber"] = df_test["Haber"].apply(str)
df_test = df_test.sample(frac = 1).reset_index(drop=True)
df_test.sample(230).sort_index()

230


,Haber,Sınıf
0,chp'den kızılay için soru önergesi chp i̇sta...,3
1,"çocuğuna ""eskilerini"" giydiriyor ünlü oyuncu...",2
2,google'da playboy tedirginliği playboy'a veri...,1
3,ankara üniversitesi tıp fakültesi farmakoloji ...,3
4,cim-bom ateş hattında : 2-3 yeşilyurt önünde...,5
...,...,...
225,"[habersağlık-i̇stanbul] diyabet, kalp hastalık...",3
226,alinur velidedeoğlu ile ayşe çavuşoğlu'nun ber...,2
227,ümi̇tler’den tek yumruk : 1-0 ay-yıldızlı ek...,5
228,artık helin'in de bir butiği olacak hülya av...,2


### Background for labeling functions

In [78]:
para = ["lira", "dolar", "döviz", "faiz"]
ekonomi_kavramlar = ["cari", "enflasyon", "merkez bankası", "imf", "kredi", "yatırım"]
sayi_birimleri = ["milyon", "milyar", "bin", "trilyon", "katrilyon"]

magazin_kavramlar = ["ünlü", "tatil", "bodrum", "konser", "sevgili", "yakışıklı", "aşk"]
magazin_meslekler = ["manken", "oyuncu", "aktör"]
magazin_kisiler = ["hülya avşar", "beren saat", "ajda pekkan", "demet akalın", "mehmet ali erbil"]

vucut_bolumleri = ["kalp", "diş", "göz", "damar", "ağız", "cilt", "vücut", "yağ"]
saglik_kavramlar = ["tedavi", "bilim", "ilaç", "teşhis", "hasta", "tıp", "ameliyat", "ssk"]
hastaliklar = ["kanser", "şeker", "yağ", "hastalık", "doğum", "anne", "görme"]

siyasi_rutbeler = ["genel başkanı", "milletvekili", "bakan", "başbakan", "cumhurbaşkanı", "dışişleri"]
siyasi_kavramlar = ["abd", "erdoğan", "kılıçdaroğlu", "demirtaş", "bahçeli", "akşener", "avrupa", "büyükşehir", "anayasa", "abd"]
siyasi_partiler = ["akp", "chp", "mhp", "hdp"]

spor_kavramlar = ["teknik", "milli", "gol", "takım", "maç", "transfer", "forma", "uefa", "direktör", "terim"]
takimlar = ["fenerbahçe", "galatasaray", "beşiktaş", "trabzonspor", "barcelona", "arjantin", "kocaelispor"]
dallar = ["futbolcu", "kupa", "saha", "hakem", "lig", "rakip", "şampiyon", "deplasman", "futbol", "basketbol", "penaltı"]

In [79]:
EKONOMI = 1
MAGAZIN = 2
SAGLIK = 3
SIYASI = 4
SPOR = 5
BELIRSIZ = -1

### Labeling functions.

In [80]:
from snorkel.labeling import labeling_function
import re

@labeling_function()
def para_iceriyor(sample_new):
    for para_tipi in para:
        if re.search(para_tipi, sample_new.Haber):
            return EKONOMI 
    return BELIRSIZ

@labeling_function()
def ekonomik_kavram_iceriyor(sample_new):
    for ekonomik_kavram in ekonomi_kavramlar:
         if re.search(ekonomik_kavram, sample_new.Haber):
            return EKONOMI
    return BELIRSIZ

@labeling_function()
def sayi_birimi_iceriyor(sample_new):
    for sayi_birimi in sayi_birimleri:
        if re.search(sayi_birimi, sample_new.Haber):
            return EKONOMI 
    return BELIRSIZ

@labeling_function()
def magazin_kavrami_iceriyor(sample_new):
    for magazin_kavrami in magazin_kavramlar:
        if re.search(magazin_kavrami, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def magazin_meslegi_iceriyor(sample_new):
    for magazin_meslegi in magazin_meslekler:
        if re.search(magazin_meslegi, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def magazin_kisiler_iceriyor(sample_new):
    for magazin_kisisi in magazin_kisiler:
        if re.search(magazin_kisisi, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def vucut_bolumu_iceriyor(sample_new):
    for vucut_bolumu in vucut_bolumleri:
        if re.search(vucut_bolumu, sample_new.Haber):
            return SAGLIK 
    return BELIRSIZ

@labeling_function()
def saglik_kavramlari_iceriyor(sample_new):
    for saglik_kavrami in saglik_kavramlar:
        if re.search(saglik_kavrami, sample_new.Haber):
            return SAGLIK
    return BELIRSIZ

@labeling_function()
def hastalik_iceriyor(sample_new):
    for hastalik in hastaliklar:
        if re.search(hastalik, sample_new.Haber):
            return SAGLIK 
    return BELIRSIZ

@labeling_function()
def siyasi_rutbe_iceriyor(sample_new):
    for siyasi_rutbe in siyasi_rutbeler:
        if re.search(siyasi_rutbe, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def siyasi_kavram_iceriyor(sample_new):
    for siyasi_kavram in siyasi_kavramlar:
        if re.search(siyasi_kavram, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def siyasi_parti_iceriyor(sample_new):
    for siyasi_parti in siyasi_partiler:
        if re.search(siyasi_parti, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def spor_kavramlar_iceriyor(sample_new):
    for spor_kavrami in spor_kavramlar:
        if re.search(spor_kavrami, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

@labeling_function()
def takimlar_iceriyor(sample_new):
    for takim in takimlar:
        if re.search(takim, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

@labeling_function()
def spor_dallari_iceriyor(sample_new):
    for spor_dali in dallar:
        if re.search(spor_dali, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

In [81]:
from snorkel.labeling import PandasLFApplier

lfs = [para_iceriyor, ekonomik_kavram_iceriyor, sayi_birimi_iceriyor, magazin_kavrami_iceriyor, magazin_meslegi_iceriyor, 
       magazin_kisiler_iceriyor, vucut_bolumu_iceriyor, saglik_kavramlari_iceriyor, hastalik_iceriyor, siyasi_rutbe_iceriyor, 
       siyasi_kavram_iceriyor, siyasi_parti_iceriyor, spor_kavramlar_iceriyor, takimlar_iceriyor, spor_dallari_iceriyor]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|███████████████████████████████████████████████████████████████████████████████| 920/920 [00:01<00:00, 814.08it/s]


In [86]:
L_test = applier.apply(df=df_test)

100%|███████████████████████████████████████████████████████████████████████████████| 230/230 [00:00<00:00, 676.30it/s]


In [82]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
para_iceriyor,0,[1],0.234783,0.233696,0.220652
ekonomik_kavram_iceriyor,1,[1],0.122826,0.121739,0.111957
sayi_birimi_iceriyor,2,[1],0.410870,0.406522,0.390217
magazin_kavrami_iceriyor,3,[2],0.547826,0.520652,0.500000
magazin_meslegi_iceriyor,4,[2],0.178261,0.169565,0.155435
magazin_kisiler_iceriyor,5,[2],0.021739,0.021739,0.013043
vucut_bolumu_iceriyor,6,[3],0.393478,0.390217,0.352174
saglik_kavramlari_iceriyor,7,[3],0.251087,0.238043,0.201087
hastalik_iceriyor,8,[3],0.266304,0.260870,0.222826
siyasi_rutbe_iceriyor,9,[4],0.265217,0.263043,0.247826


In [83]:
L_train.shape

(920, 15)

### Convert the labels from LFs into a single noise-aware probabilistic label per data point

In [84]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter(6)
preds_train = majority_model.predict(L=L_train)

In [85]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=6, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [88]:
Y_test = df_test.Sınıf.values

majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")
      
label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   76.1%
Label Model Accuracy:     72.6%


The **majority vote model** or more **sophisticated LabelModel** could in principle be used directly as a classifier if the outputs of our labeling functions were made available at test time. However, these models (i.e. these re-weighted combinations of our labeling function's votes) will *abstain on the data points that our labeling functions don't cover* (and additionally, may require slow or unavailable features to execute at test time).

In [90]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L=L_train)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)

## Training a Classifier

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 5))
X_train = vectorizer.fit_transform(df_train_filtered.Haber.tolist())
X_test = vectorizer.transform(df_test.Haber.tolist())

In [94]:
from snorkel.utils import probs_to_preds

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

In [95]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver="liblinear")
sklearn_model.fit(X=X_train, y=preds_train_filtered)

D:\ProgramFiles\Anaconda3\envs\snorkel\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [96]:
print(f"Test Accuracy: {sklearn_model.score(X=X_test, y=Y_test) * 100:.1f}%")

Test Accuracy: 74.8%
